In [1]:
from google.colab import drive, files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install catalyst

In [3]:
import os

from pathlib import Path

import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from PIL import Image
from torch.utils.data import Dataset, DataLoader ,random_split
from torchvision import datasets, models, transforms

from catalyst import dl
from catalyst.dl.callbacks import AccuracyCallback, EarlyStoppingCallback
from catalyst.utils import metrics

In [4]:
ROOT_DIR = '/content/drive/My Drive/Colab Notebooks/'
SUBMISSION_FILE = 'submission.csv'
MODEL_PATH = 'model.pkt'
NUM_CLASSES = 38
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(DEVICE)

cuda


In [5]:
class MultiClassDataset(Dataset):

    def __init__(self , csv_file , img_dir , transform=None):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
    
    def __getitem__(self, idx):
        d = self.df.iloc[idx]
        image = Image.open(f'{self.img_dir}/{d.Name}').convert("RGB")
        label = torch.tensor(d[1:].tolist() , dtype=torch.float32)
    
        if self.transform is not None:
            image = self.transform(image)
        return image, label
  
    def __len__(self):
        return len(self.df)

In [6]:
batch_size = 32
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset = MultiClassDataset(ROOT_DIR + 'data/training_labels.csv' , ROOT_DIR + 'data/training_images' , transform)

number_of_validation_samples = int(len(dataset)*0.10) 
training_set, validation_set  = random_split(dataset , [len(dataset) - number_of_validation_samples, number_of_validation_samples])

print(f'Number of samples: Train: {len(training_set)}, Validation: {len(validation_set)}')
loaders = {
    "train":DataLoader(training_set , shuffle=True, batch_size=batch_size),
    "valid": DataLoader(validation_set , shuffle=True, batch_size=batch_size)
    }

Number of samples: Train: 3733, Validation: 414


In [7]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def get_classification_network(in_features, num_classes):
    return nn.Sequential(
         nn.Linear(in_features, in_features // 2),
         # BatchNorm
         nn.ReLU(),
         # Dropout(p = 0.5)
         nn.Linear(in_features // 2, in_features // 4),
         nn.ReLU(),
         nn.Linear(in_features // 4, in_features // 8),
         nn.ReLU(),
         nn.Linear(in_features // 8, num_classes)
    )

def get_model(num_classes, feature_extract=True):
    model = models.resnext101_32x8d(pretrained = True)
    set_parameter_requires_grad(model, feature_extract)
    num_ftrs = model.fc.in_features
    model.fc = get_classification_network(num_ftrs, num_classes)
    return model

def skyhacks_f1_score(preds, y):
    return np.mean([metrics.f1_score(preds[:, i], y[:, i])[0].item() for i in range(preds.shape[0])])

In [8]:
model = get_model(NUM_CLASSES)

In [9]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
callbacks = [
    EarlyStoppingCallback(patience=5)
]
num_epochs = 100

class CustomRunner(dl.Runner):
    
    def predict_batch(self, batch):
        x, y = batch
        return self.model(x.to(self.device))
    
    def _handle_batch(self, batch):
        x, y = batch
        output = self.model(x)
        loss = self.state.criterion(output, y)

        preds = torch.sigmoid(output).data > 0.5
        preds = preds.to(torch.float32)

        f1 = skyhacks_f1_score(preds, y)
        self.batch_metrics = {
            "loss": loss,
            "f1-score": f1
        }
        
        if self.state.is_train_loader:
            loss.backward()
            self.state.optimizer.step()
            self.state.optimizer.zero_grad()           


In [10]:
runner = CustomRunner()

runner.train(
    loaders=loaders,
    model=model, 
    criterion=criterion, 
    optimizer=optimizer,
    callbacks=callbacks,
    num_epochs=num_epochs, 
    logdir="./logs", 
    verbose=True
)

1/100 * Epoch (train): 100% 117/117 [02:54<00:00,  1.49s/it, f1-score=0.941, loss=0.207]
1/100 * Epoch (valid): 100% 13/13 [00:19<00:00,  1.53s/it, f1-score=0.949, loss=0.208]
[2020-11-14 11:30:23,873] 
1/100 * Epoch 1 (train): f1-score=0.9400 | loss=0.2505
1/100 * Epoch 1 (valid): f1-score=0.9455 | loss=0.2124
2/100 * Epoch (train): 100% 117/117 [02:55<00:00,  1.50s/it, f1-score=0.944, loss=0.190]
2/100 * Epoch (valid): 100% 13/13 [00:19<00:00,  1.50s/it, f1-score=0.947, loss=0.200]
[2020-11-14 11:34:05,818] 
2/100 * Epoch 2 (train): f1-score=0.9516 | loss=0.1982
2/100 * Epoch 2 (valid): f1-score=0.9520 | loss=0.2001
3/100 * Epoch (train): 100% 117/117 [02:58<00:00,  1.52s/it, f1-score=0.951, loss=0.185]
3/100 * Epoch (valid): 100% 13/13 [00:19<00:00,  1.48s/it, f1-score=0.952, loss=0.203]
[2020-11-14 11:37:47,899] 
3/100 * Epoch 3 (train): f1-score=0.9529 | loss=0.1886
3/100 * Epoch 3 (valid): f1-score=0.9551 | loss=0.1920
4/100 * Epoch (train): 100% 117/117 [02:56<00:00,  1.51s/it, 

In [11]:
torch.save(runner.model, MODEL_PATH)
files.download(MODEL_PATH)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>